In [16]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from keras.callbacks import EarlyStopping

In [17]:
data = pd.read_csv('fer2013.csv')
print("Dataset loaded successfully.")

Dataset loaded successfully.


In [18]:
print(data.head())
print(data.info())
print("\nEmotion distribution:\n", data['emotion'].value_counts())
print("\nUsage distribution:\n", data['Usage'].value_counts())

   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35887 entries, 0 to 35886
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   emotion  35887 non-null  int64 
 1   pixels   35887 non-null  object
 2   Usage    35887 non-null  object
dtypes: int64(1), object(2)
memory usage: 841.2+ KB
None

Emotion distribution:
 emotion
3    8989
6    6198
4    6077
2    5121
0    4953
5    4002
1     547
Name: count, dtype: int64

Usage distribution:
 Usage
Training       28709
PublicTest      3589
PrivateTest     3589

In [19]:
img_size = 48
# Convert pixel string to numpy array
def preprocess_pixels(pixel_string):
    pixels = np.array(pixel_string.split(' '), dtype='float32')
    return pixels.reshape(img_size, img_size, 1)

data['pixels'] = data['pixels'].apply(preprocess_pixels)


In [20]:
emotion_labels = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Sad', 5: 'Surprise', 6: 'Neutral'}
num_classes = len(emotion_labels)
# Convert numerical labels to one-hot encoded vectors [1, 7]
data['emotion_one_hot'] = data['emotion'].apply(lambda x: to_categorical(x, num_classes=num_classes))

In [21]:
#  Data Splitting
train_df = data[data['Usage'] == 'Training']
test_df = data[data['Usage'] == 'PublicTest']
val_df = data[data['Usage'] == 'PrivateTest']

#  features (X) and labels (y) for each set
X_train = np.stack(train_df['pixels'].values)
y_train = np.stack(train_df['emotion_one_hot'].values)

X_val = np.stack(val_df['pixels'].values)
y_val = np.stack(val_df['emotion_one_hot'].values)

X_test = np.stack(test_df['pixels'].values)
y_test = np.stack(test_df['emotion_one_hot'].values)

print(f"\nTraining data shape: {X_train.shape}, labels shape: {y_train.shape}")
print(f"Validation data shape: {X_val.shape}, labels shape: {y_val.shape}")
print(f"Test data shape: {X_test.shape}, labels shape: {y_test.shape}")



Training data shape: (28709, 48, 48, 1), labels shape: (28709, 7)
Validation data shape: (3589, 48, 48, 1), labels shape: (3589, 7)
Test data shape: (3589, 48, 48, 1), labels shape: (3589, 7)


In [22]:
model = Sequential(name="Emotion_Detection_CNN")

# Block 1
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Block 2
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Block 3
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flatten the feature maps to feed into the Dense layers
model.add(Flatten())

# Fully Connected Layers
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Output Layer
model.add(Dense(7, activation='softmax'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [23]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
BATCH_SIZE = 64
EPOCHS = 100

early_stopping = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    verbose=1,
    restore_best_weights=True
)

print("\n--- Starting Model Training ---")


history = model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping],
    verbose=1
)

print("--- Model Training Complete ---\n")


--- Starting Model Training ---
Epoch 1/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 318s 691ms/step - accuracy: 0.2828 - loss: 2.2002 - val_accuracy: 0.4227 - val_loss: 1.4889
Epoch 2/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 307s 659ms/step - accuracy: 0.4292 - loss: 1.4994 - val_accuracy: 0.4539 - val_loss: 1.4188
Epoch 3/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 315s 644ms/step - accuracy: 0.4814 - loss: 1.3630 - val_accuracy: 0.4990 - val_loss: 1.2905
Epoch 4/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 322s 643ms/step - accuracy: 0.5108 - loss: 1.2835 - val_accuracy: 0.5118 - val_loss: 1.2762
Epoch 5/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 320s 639ms/step - accuracy: 0.5253 - loss: 1.2471 - val_accuracy: 0.5258 - val_loss: 1.2246
Epoch 6/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 297s 661ms/step - accuracy: 0.5485 - loss: 1.1911 - val_accuracy: 0.4893 - val_loss: 1.3388
Epoch 7/100
449/449 ━━━━━━━━━━━━━━━━━━━━ 320s 658ms/step - accuracy: 0.5610 - loss: 1.1564 - val_accuracy: 0.5425 - val_loss: 1.2139
Epoch 8/100
449/449 ━━━━━━━━━━━━━━━━

In [ ]:
print("\nEvaluating model on the test set...")
test_loss,test_accuracy = model.evaluate(X_test, y_test, verbose=1)

print(f"\nTest Loss: {test_loss:.2f}")
print(f"Test Accuracy: {test_accuracy*100:.2f}%")


In [ ]:
print("\n--- Evaluating Model on Public Test Set ---")
public_test_loss, public_test_accuracy = model.evaluate(X_val, y_val, verbose=0)
print(f"Model Accuracy on Public Test Set: {public_test_accuracy*100:.2f}%")

In [ ]:
# Select a random index from the validation set
random_index = np.random.randint(0, len(X_test))

# Get the random image and its one-hot encoded label
random_image = X_test[random_index]
actual_emotion_one_hot = y_test[random_index]

# ✨ FIX: Convert the one-hot encoded array to a single integer label
actual_emotion_label = np.argmax(actual_emotion_one_hot)
actual_emotion_name = emotion_labels[actual_emotion_label]

# Reshape the image to make a batch of size 1 for prediction
random_image_for_prediction = np.expand_dims(random_image, axis=0)

# Make a prediction
predicted_probabilities = model.predict(random_image_for_prediction)
predicted_emotion_label = np.argmax(predicted_probabilities)
predicted_emotion_name = emotion_labels[predicted_emotion_label]

# Display the image and the results
plt.figure(figsize=(4, 4))
plt.imshow(random_image.reshape(48, 48), cmap='gray')
plt.title(f"Actual: {actual_emotion_name}\nPredicted: {predicted_emotion_name}")
plt.axis('off')
plt.show()


